# Model Alignment Process

### Import necessary modules

In [1]:
import pandas as pd
from tabulate import tabulate

### Load excel files

In [2]:
# Read excel files
orderstab_df = pd.read_excel('Cleansed_Sample - Superstore.xlsx', sheet_name='Orders')
peopletab_df = pd.read_excel('Cleansed_Sample - Superstore.xlsx', sheet_name='People')
returnstab_df = pd.read_excel('Cleansed_Sample - Superstore.xlsx', sheet_name='Returns')

In [3]:
orderstab_df.shape

(9994, 20)

In [4]:
orderstab_dup = orderstab_df[orderstab_df['Product ID'].isin(['FUR-BO-10002213', 'FUR-CH-10001146', 'FUR-FU-10001473', 'FUR-FU-10004017', 
                                                              'FUR-FU-10004091', 'FUR-FU-10004270', 'FUR-FU-10004848', 'FUR-FU-10004864',
                                                              'OFF-AP-10000576', 'OFF-AR-10001149', 'OFF-BI-10002026', 'OFF-BI-10004632',
                                                              'OFF-BI-10004654', 'OFF-PA-10000357', 'OFF-PA-10000477', 'OFF-PA-10000659',
                                                              'OFF-PA-10001166', 'OFF-PA-10001970', 'OFF-PA-10002195', 'OFF-PA-10002377',
                                                              'OFF-PA-10003022', 'OFF-ST-10001228', 'OFF-ST-10004950', 'TEC-AC-10002049',
                                                              'TEC-AC-10002550', 'TEC-AC-10003832', 'TEC-MA-10001148', 'TEC-PH-10001530',
                                                              'TEC-PH-10001795', 'TEC-PH-10002200', 'TEC-PH-10002310', 'TEC-PH-10004531'])]

orderstab_dup = orderstab_dup.sort_values(by=['Product ID', 'Product Name'])
counter = 0
idx_set = set()
for idx, row in orderstab_dup[['Product ID', 'Product Name']].iterrows():
    product_id = row['Product ID']
    product_name = row['Product Name']

    idx_set.add(idx)
    for idx_check, row_check in orderstab_dup[['Product ID', 'Product Name']].iterrows():
        product_id_check = row_check['Product ID']
        product_name_check = row_check['Product Name']
        
        if (product_id == product_id_check) and (product_name != product_name_check) and (idx_check not in idx_set):
            orderstab_dup.at[idx_check, 'Product ID'] = f"{product_id_check}-DUP"
            print(idx, ' - ', product_id, ' - ', product_name, ' ------- ', idx_check, ' - ', product_id_check, ' - ', product_name_check)
            print('___________________________________________________________________________________________________________________________')
            counter+=1
            if counter == 50:
                break
            print(idx_check)
            idx_set.add(idx_check)

2115  -  FUR-BO-10002213  -  DMI Eclipse Executive Suite Bookcases  -------  2471  -  FUR-BO-10002213  -  Sauder Forest Hills Library, Woodland Oak Finish
___________________________________________________________________________________________________________________________
2471
2115  -  FUR-BO-10002213  -  DMI Eclipse Executive Suite Bookcases  -------  2808  -  FUR-BO-10002213  -  Sauder Forest Hills Library, Woodland Oak Finish
___________________________________________________________________________________________________________________________
2808
2115  -  FUR-BO-10002213  -  DMI Eclipse Executive Suite Bookcases  -------  5079  -  FUR-BO-10002213  -  Sauder Forest Hills Library, Woodland Oak Finish
___________________________________________________________________________________________________________________________
5079
2115  -  FUR-BO-10002213  -  DMI Eclipse Executive Suite Bookcases  -------  8712  -  FUR-BO-10002213  -  Sauder Forest Hills Library, Woodland Oak 

In [5]:
#orderstab_dup[orderstab_dup['Product ID'].isin(['FUR-BO-10002213', 'FUR-BO-10002213-DUP'])]
orderstab_df = orderstab_df[~orderstab_df['Product ID'].isin(['FUR-BO-10002213', 'FUR-CH-10001146', 'FUR-FU-10001473', 'FUR-FU-10004017', 
                                                              'FUR-FU-10004091', 'FUR-FU-10004270', 'FUR-FU-10004848', 'FUR-FU-10004864',
                                                              'OFF-AP-10000576', 'OFF-AR-10001149', 'OFF-BI-10002026', 'OFF-BI-10004632',
                                                              'OFF-BI-10004654', 'OFF-PA-10000357', 'OFF-PA-10000477', 'OFF-PA-10000659',
                                                              'OFF-PA-10001166', 'OFF-PA-10001970', 'OFF-PA-10002195', 'OFF-PA-10002377',
                                                              'OFF-PA-10003022', 'OFF-ST-10001228', 'OFF-ST-10004950', 'TEC-AC-10002049',
                                                              'TEC-AC-10002550', 'TEC-AC-10003832', 'TEC-MA-10001148', 'TEC-PH-10001530',
                                                              'TEC-PH-10001795', 'TEC-PH-10002200', 'TEC-PH-10002310', 'TEC-PH-10004531'])]

orderstab_df = pd.concat([orderstab_df, orderstab_dup], ignore_index=True)
orderstab_df.shape

(9994, 20)

### Split the tables based on the ERD

In [6]:
Orders = orderstab_df[['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Country/Region', 'City', 'State', 'Postal Code']]
Order_Details = orderstab_df[['Order ID', 'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit']]
Products = orderstab_df[['Product ID', 'Sub-Category', 'Product Name']]
Sub_Categories = orderstab_df[['Category', 'Sub-Category']]
Categories = orderstab_df[['Category']]
Returns = returnstab_df[['Order ID']]
Ship_Mode = orderstab_df[['Ship Mode']]
Customers = orderstab_df[['Customer ID', 'Customer Name', 'Segment']]
Customer_Address = orderstab_df[['Country/Region', 'City', 'State', 'Postal Code', 'Region']]
Address_Customer = orderstab_df[['Customer ID', 'Country/Region', 'City', 'State', 'Postal Code', 'Region']]
Segment = orderstab_df[['Segment']]
Region = orderstab_df[['Region']]
Regional_Managers = peopletab_df[['Regional Manager', 'Region']]

In [7]:
Customers_copy = Customers.copy()
split_customer_name = Customers['Customer Name'].str.split(expand=True, n=1)
Customers_copy.loc[:, 'First Name'] = split_customer_name[0]
Customers_copy.loc[:, 'Last Name'] = split_customer_name[1]
Customers_reordered = ['Customer ID', 'First Name', 'Last Name', 'Segment']
Customers_copy = Customers_copy[Customers_reordered]
Customers = Customers_copy

In [8]:
Regional_Managers_copy = Regional_Managers.copy()
split_name = Regional_Managers['Regional Manager'].str.split(expand=True, n=1)
Regional_Managers_copy.loc[:, 'First Name'] = split_name[0]
Regional_Managers_copy.loc[:, 'Last Name'] = split_name[1]
Regional_Managers_reordered = ['First Name', 'Last Name', 'Region']
Regional_Managers_copy = Regional_Managers_copy[Regional_Managers_reordered]
Regional_Managers = Regional_Managers_copy

### Save the new tables in an excel file

In [9]:
excel_filename = 'Group6_ModelAlignment_Initial.xlsx'

with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
    Orders.to_excel(writer, sheet_name='Orders', index=False)
    Order_Details.to_excel(writer, sheet_name='Order_Details', index=False)
    Products.to_excel(writer, sheet_name='Products', index=False)
    Sub_Categories.to_excel(writer, sheet_name='Sub_Categories', index=False)
    Categories.to_excel(writer, sheet_name='Categories', index=False)
    Returns.to_excel(writer, sheet_name='Returns', index=False)
    Ship_Mode.to_excel(writer, sheet_name='Ship_Mode', index=False)
    Customers.to_excel(writer, sheet_name='Customers', index=False)
    Customer_Address.to_excel(writer, sheet_name='Customer_Address', index=False)
    Address_Customer.to_excel(writer, sheet_name='Address_Customer', index=False)
    Segment.to_excel(writer, sheet_name='Segment', index=False)
    Region.to_excel(writer, sheet_name='Region', index=False)
    Regional_Managers.to_excel(writer, sheet_name='Regional_Managers', index=False)

### Drop the duplicates from the new tables formed and save each table separately in a CSV file

In [10]:
# Load the Excel file
excel_file_name = 'Group6_ModelAlignment_Initial.xlsx'
excel_file = pd.ExcelFile(excel_file_name)

# Create a list to store the results
results = []

# Create an Excel writer object to save the cleaned data to a new file
output_excel_file = 'Group6_ModelAlignment_Cleaned.xlsx'

with pd.ExcelWriter(output_excel_file, engine='openpyxl') as writer:
    # Loop through each sheet in the Excel file
    for sheet in excel_file.sheet_names:
        # Read the current sheet
        df = pd.read_excel(excel_file, sheet_name=sheet)

        # Get row count before removing duplicates
        initial_row_count = df.shape[0]

        # Check and display the number of duplicate rows
        duplicate_count = df.duplicated().sum()

        # Drop the duplicates
        df_cleaned = df.drop_duplicates()

        # Get row count after removing duplicates
        final_row_count = df_cleaned.shape[0]

        # Append the results for tabulation
        results.append([sheet, initial_row_count, duplicate_count, final_row_count])

        # Save the cleaned sheet into the new Excel file
        df_cleaned.to_excel(writer, sheet_name=sheet, index=False)

        # Save the cleaned sheet as a CSV file
        csv_output_file = f'{sheet}_Cleaned.csv'
        df_cleaned.to_csv(csv_output_file, index=False)
        print(f"Cleaned sheet '{sheet}' saved to CSV file: {csv_output_file}")

    # Create a summary DataFrame for tabulated results
    summary_df = pd.DataFrame(results, columns=['Sheet Name', 'Rows Before', 'Duplicates Found', 'Rows After'])

    # Write the summary results to a new sheet
    summary_df.to_excel(writer, sheet_name='Summary', index=False)

# Print the tabulated results to the console
headers = ['Sheet Name', 'Rows Before', 'Duplicates Found', 'Rows After']
print(tabulate(results, headers=headers, tablefmt="grid"))

print(f"Cleaned data and summary have been saved to '{output_excel_file}'")

Cleaned sheet 'Orders' saved to CSV file: Orders_Cleaned.csv
Cleaned sheet 'Order_Details' saved to CSV file: Order_Details_Cleaned.csv
Cleaned sheet 'Products' saved to CSV file: Products_Cleaned.csv
Cleaned sheet 'Sub_Categories' saved to CSV file: Sub_Categories_Cleaned.csv
Cleaned sheet 'Categories' saved to CSV file: Categories_Cleaned.csv
Cleaned sheet 'Returns' saved to CSV file: Returns_Cleaned.csv
Cleaned sheet 'Ship_Mode' saved to CSV file: Ship_Mode_Cleaned.csv
Cleaned sheet 'Customers' saved to CSV file: Customers_Cleaned.csv
Cleaned sheet 'Customer_Address' saved to CSV file: Customer_Address_Cleaned.csv
Cleaned sheet 'Address_Customer' saved to CSV file: Address_Customer_Cleaned.csv
Cleaned sheet 'Segment' saved to CSV file: Segment_Cleaned.csv
Cleaned sheet 'Region' saved to CSV file: Region_Cleaned.csv
Cleaned sheet 'Regional_Managers' saved to CSV file: Regional_Managers_Cleaned.csv
+-------------------+---------------+--------------------+--------------+
| Sheet Name 